In [4]:
using Einsum
using StaticArrays
using EtherArrays
using KernelAbstractions
using oneAPI

In [5]:
@kernel function ker!(data)
    idx::Int = @index(Global)
    a = E2SquareMatrix{2}(idx, 1, data)
    b = E2SquareMatrix{2}(idx, 5, data)
    c = similar(a)
    c .= a
    @einsum a[i, j] = c[i, k] * b[k, j]
end

ker! (generic function with 4 methods)

In [3]:
data_cpu = randn(Float32, 3, 8);
data_gpu = oneArray(data_cpu)

3×8 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
 -0.0814797  -0.857169  1.14573   …  1.45512     0.255798    0.91113
 -1.05177     1.81526   0.343711     0.0155039   0.0461652  -1.95977
 -0.604466    0.470063  0.642597     0.690566   -0.205334   -0.381764

In [6]:
ker!(oneAPIBackend(), 3)(data_gpu, ndrange=(3,))
KernelAbstractions.synchronize(oneAPIBackend())

In [7]:
data_gpu

3×8 oneArray{Float32, 2, oneAPI.oneL0.DeviceBuffer}:
 1.7428     1.33989    1.02307   …  1.45512     0.255798    0.91113
 0.215341  -0.379801  -0.722151     0.0155039   0.0461652  -1.95977
 0.412498   0.639567  -0.121203     0.690566   -0.205334   -0.381764